In [1]:
import numpy as np
import uproot
import datetime as dt
from sndaq.analysis import AnalysisHandler, AnalysisConfig
from sndaq.trigger import TriggerHandler

## Setup Analyses and Storage

In [2]:
def nstimes_to_hrtimes(ns_time, year=None):
    """Convert ns since start of year times to human readable (hr) times
    
    ns_time : numpy.array
        Array of times, measured since 01/01/`year` 00:00:00.000 with nanosecond precision
    year : int
        Year corresponding to ns times
        
    Returns
    -------
    dt_start : np.datetime64
        datetime corresponding to start of ns_times (run_start)
    nt_time_norm : numpy.array
        Time bins relative to dt_start in units s, should be uniform 0.5s
    """
    if year is None:
        year = dt.datetime.now().year
    dt_start = np.datetime64(f'{year}-01-01') + np.timedelta64(ns_time[0], 'ns')
    ns_time_norm = (ns_time - ns_time[0]) / 1e9      
    return dt_start, ns_time_norm


def get_500ms_rundata(file, year=None):
    """Get 500ms rundata from sndaq .root file
    
    Parameters
    ----------
    file : string or Pathlike
        Path to SNDAQ data file
    year : int
        Year of run during which data was collected
        
    Returns
    -------
    data : np.ndarray with Dim(n_500ms_bins, n_doms)
        500ms data for all doms
    start : np.datetime64
        Datetime corresponding to start of data 
    time_bins : numpy.array
        Time bins relative to start in units s, should be uniform 0.5s
    """
    with uproot.open(file) as f:
        sn_all = f['sn_all/sn_all_data']
        data = sn_all['data'].array().to_numpy()  # Might hang a little
        gps_time_high = sn_all['SN_Streaming_t/gps_time_high'].array().to_numpy() # First 32 bits of times
        gps_time_low = sn_all['SN_Streaming_t/gps_time_low'].array().to_numpy()  # Last 32 bits of times
        gps_time = np.bitwise_or(np.left_shift(gps_time_high, 32), gps_time_low)
    start, time = nstimes_to_hrtimes(gps_time, year)
    
    return data, start, time

In [3]:
sndata_file = '../../scratch/data/rundata/SPS/sndata_137104_000.root'
data, start, time = get_500ms_rundata(sndata_file)

idc_dropped = np.where(np.all(data==0, axis=0))[0]
data = np.delete(data, idc_dropped, axis=1)
n_dropped_doms = idc_dropped.size

data.shape

(57608, 5075)

In [4]:
ndom = 5160 - n_dropped_doms
ana_conf = AnalysisConfig.from_config(conf_path='../..data/config/analysis.config')
ana = AnalysisHandler(config=ana_conf, dropped_doms=idc_dropped)

nbins_window = 400  # 200 s of rates in 0.5s bins
nbins_trigger = int((ana.duration_nosearch + 3*max(ana._binnings)) / 500 - 1)
nbins_fill_trailing = nbins_trigger - int(ana.duration_nosearch/2 /500)
nbins_fill_leading = nbins_trigger - nbins_fill_trailing - nbins_window

dom_bkg_mu = 140
dom_bkg_sig = 22

rate_500ms = np.zeros(shape=(nbins_window, ndom), dtype=np.uint16)
signi = np.zeros(shape=(nbins_window, len(ana.analyses)), dtype=float)

filler = None
np.random.seed(42)

## Feeding 500 ms data to PySNDAQ

In [7]:
# Define a fresh instance for convenience of repeatedly running this cell
ana = AnalysisHandler(config=ana_conf, dropped_doms=idc_dropped)
alert = TriggerHandler()

i = 1
for idx_time, rate in enumerate(data):

    ana.buffer_analysis.append(rate.astype(np.uint32))
    ana.update_analyses()
    ana.process_triggers()

    if ana.trigger_finalized:
        print(f"Candidate: {i}")
        i += 1
        alert.process_trigger(ana.current_trigger)
        ana.current_trigger.reset()

Candidate: 1
Basic Trigger Processing...
xi=5.019 in   0.5 +(0.0) s Analysis @t=108.0 s
Candidate: 2
Basic Trigger Processing...
xi=4.645 in   0.5 +(0.0) s Analysis @t=296.0 s
Candidate: 3
Basic Trigger Processing...
xi=5.106 in   0.5 +(0.0) s Analysis @t=329.0 s
Candidate: 4
Basic Trigger Processing...
xi=4.639 in   1.5 +(0.0) s Analysis @t=601.5 s
Candidate: 5
Basic Trigger Processing...
xi=4.120 in   1.5 +(0.0) s Analysis @t=913.5 s
Candidate: 6
Basic Trigger Processing...
xi=4.395 in   1.5 +(1.0) s Analysis @t=998.0 s
Candidate: 7
Basic Trigger Processing...
xi=4.361 in   1.5 +(0.0) s Analysis @t=1071.0 s
Candidate: 8
Basic Trigger Processing...
xi=5.184 in   0.5 +(0.0) s Analysis @t=1471.0 s
Candidate: 9
Basic Trigger Processing...
xi=4.523 in  10.0 +(0.5) s Analysis @t=1589.5 s
Candidate: 10
Basic Trigger Processing...
xi=4.046 in   1.5 +(0.5) s Analysis @t=1652.5 s
Candidate: 11
Basic Trigger Processing...
xi=5.030 in   0.5 +(0.0) s Analysis @t=1956.0 s
Candidate: 12
Basic Trigg

Candidate: 92
Basic Trigger Processing...
xi=4.708 in   4.0 +(3.5) s Analysis @t=12524.5 s
Candidate: 93
Basic Trigger Processing...
xi=4.103 in   1.5 +(0.5) s Analysis @t=12718.0 s
Candidate: 94
Basic Trigger Processing...
xi=4.436 in   0.5 +(0.0) s Analysis @t=12812.0 s
Candidate: 95
Basic Trigger Processing...
xi=4.609 in   0.5 +(0.0) s Analysis @t=12843.5 s
Candidate: 96
Basic Trigger Processing...
xi=4.254 in   1.5 +(0.5) s Analysis @t=12887.5 s
Candidate: 97
Basic Trigger Processing...
xi=4.234 in   0.5 +(0.0) s Analysis @t=13441.0 s
Candidate: 98
Basic Trigger Processing...
xi=4.529 in  10.0 +(8.5) s Analysis @t=13461.5 s
Candidate: 99
Basic Trigger Processing...
xi=4.187 in  10.0 +(2.0) s Analysis @t=14018.0 s
Candidate: 100
Basic Trigger Processing...
xi=4.652 in   1.5 +(0.5) s Analysis @t=14167.0 s
Candidate: 101
Basic Trigger Processing...
xi=4.231 in   1.5 +(0.5) s Analysis @t=14285.5 s
Candidate: 102
Basic Trigger Processing...
xi=4.582 in   1.5 +(0.0) s Analysis @t=14326.

Candidate: 182
Basic Trigger Processing...
xi=4.187 in   0.5 +(0.0) s Analysis @t=26014.0 s
Candidate: 183
Basic Trigger Processing...
xi=4.159 in   0.5 +(0.0) s Analysis @t=26050.0 s
Candidate: 184
Basic Trigger Processing...
xi=5.116 in   1.5 +(1.0) s Analysis @t=26193.5 s
Candidate: 185
Basic Trigger Processing...
xi=4.482 in   1.5 +(0.5) s Analysis @t=26447.5 s
Candidate: 186
Basic Trigger Processing...
xi=4.134 in   0.5 +(0.0) s Analysis @t=26506.5 s
Candidate: 187
Basic Trigger Processing...
xi=4.091 in   0.5 +(0.0) s Analysis @t=26639.0 s
Candidate: 188
Basic Trigger Processing...
xi=4.014 in   1.5 +(1.0) s Analysis @t=26717.0 s
Candidate: 189
Basic Trigger Processing...
xi=4.372 in   0.5 +(0.0) s Analysis @t=26837.0 s
Candidate: 190
Basic Trigger Processing...
xi=4.095 in   0.5 +(0.0) s Analysis @t=27093.0 s
Candidate: 191
Basic Trigger Processing...
xi=5.455 in   0.5 +(0.0) s Analysis @t=27180.0 s
Candidate: 192
Basic Trigger Processing...
xi=4.317 in   0.5 +(0.0) s Analysis @

In [8]:
for i, search in enumerate(ana.analyses):
    idc_debug = {
        'eod': search.idx_eod,
        'bgl': search.idx_bgl,
        'exl': search.idx_exl,
        'sw':  search.idx_sw,
        'ext': search.idx_ext,
        'bgt': search.idx_bgt
    }
    
    print(f"\nSearch #{i:<2d} :{search.binsize/1e3:4.1f} +{search.offset/1e3:<4.1f}s")
    for key, val in idc_debug.items():
        print(f"{key:>10s} : {val:<5d}")


Search #0  : 0.5 +0.0 s
       eod : 1379 
       bgl : 779  
       exl : 719  
        sw : 718  
       ext : 658  
       bgt : 58   

Search #1  : 1.5 +0.0 s
       eod : 1379 
       bgl : 779  
       exl : 719  
        sw : 716  
       ext : 656  
       bgt : 56   

Search #2  : 1.5 +0.5 s
       eod : 1378 
       bgl : 778  
       exl : 718  
        sw : 715  
       ext : 655  
       bgt : 55   

Search #3  : 1.5 +1.0 s
       eod : 1377 
       bgl : 777  
       exl : 717  
        sw : 714  
       ext : 654  
       bgt : 54   

Search #4  : 4.0 +0.0 s
       eod : 1379 
       bgl : 779  
       exl : 719  
        sw : 711  
       ext : 651  
       bgt : 51   

Search #5  : 4.0 +0.5 s
       eod : 1378 
       bgl : 778  
       exl : 718  
        sw : 710  
       ext : 650  
       bgt : 50   

Search #6  : 4.0 +1.0 s
       eod : 1377 
       bgl : 777  
       exl : 717  
        sw : 709  
       ext : 649  
       bgt : 49   

Search #7  : 4.0 +1.5 s
  

In [16]:
for i, search in enumerate(ana.analyses):
    idc_debug = {
        'add_bgl': search._idx_addbgl[[0,-1]],
        'sub_bgl': search._idx_subbgl[[0,-1]],
        'add_sw': search._idx_addsw[[0,-1]],
        'sub_sw': search._idx_subsw[[0,-1]],
        'add_bgt': search._idx_addbgt[[0,-1]],
        'sub_bgt':  search._idx_subbgt[[0,-1]],
    }
    
    print(f"\nSearch #{i:<2d} :{search.binsize/1e3:4.1f} +{search.offset/1e3:<4.1f}s")
    for key, val in idc_debug.items():
        print(f"{key:>10s} : {val[0]:>5d} .. {val[1]:<5d}")


Search #0  : 0.5 +0.0 s
   add_bgl :  1378 .. 1378 
   sub_bgl :   778 .. 778  
    add_sw :   718 .. 718  
    sub_sw :   717 .. 717  
   add_bgt :   657 .. 657  
   sub_bgt :    57 .. 57   

Search #1  : 1.5 +0.0 s
   add_bgl :  1376 .. 1378 
   sub_bgl :   776 .. 778  
    add_sw :   716 .. 718  
    sub_sw :   713 .. 715  
   add_bgt :   653 .. 655  
   sub_bgt :    53 .. 55   

Search #2  : 1.5 +0.5 s
   add_bgl :  1375 .. 1377 
   sub_bgl :   775 .. 777  
    add_sw :   715 .. 717  
    sub_sw :   712 .. 714  
   add_bgt :   652 .. 654  
   sub_bgt :    52 .. 54   

Search #3  : 1.5 +1.0 s
   add_bgl :  1374 .. 1376 
   sub_bgl :   774 .. 776  
    add_sw :   714 .. 716  
    sub_sw :   711 .. 713  
   add_bgt :   651 .. 653  
   sub_bgt :    51 .. 53   

Search #4  : 4.0 +0.0 s
   add_bgl :  1371 .. 1378 
   sub_bgl :   771 .. 778  
    add_sw :   711 .. 718  
    sub_sw :   703 .. 710  
   add_bgt :   643 .. 650  
   sub_bgt :    43 .. 50   

Search #5  : 4.0 +0.5 s
   add_bgl

In [6]:
for i, (analysis, xi) in enumerate(zip(ana.analyses, signi.T), start=1):
    print(f"\nSearch #{i:<2d}: {analysis.binsize/1e3:<4.1f} +{analysis.offset/1e3:<4.1f}s\n"
          f"   max xi : {xi.max():4.2f}  @ bin t={xi.argmax()*0.5}s from window start")


Search #1 : 1.0  +0.0 s
   max xi : 2.70  @ bin t=36.5s from window start

Search #2 : 1.0  +0.5 s
   max xi : 2.54  @ bin t=8.5s from window start
